In [1]:
import nltk

from os import getcwd
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

In [2]:
nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Winzyl\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Winzyl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import json

with open('C:/Users/Winzyl/Desktop/migrate/fildata/positive-FINAL.json') as f:
  pos = json.load(f)

with open('C:/Users/Winzyl/Desktop/migrate/fildata/negative-FINAL.json') as f:
  neg = json.load(f)

In [4]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [5]:
stopwords_filipino = ["akin","aking","ako","alin","am","amin","aming","ang","ano","anumang","apat","at","atin","ating","ay","bababa","bago","bakit","bawat","bilang","dahil","dalawa","dapat","din","dito","doon","gagawin","gayunman","ginagawa","ginawa","ginawang","gumawa","gusto","habang","hanggang","hindi","huwag","iba","ibaba","ibabaw","ibig","ikaw","ilagay","ilalim","ilan","inyong","isa","isang","itaas","ito","iyo","iyon","iyong","ka","kahit","kailangan","kailanman","kami","kanila","kanilang","kanino","kanya","kanyang","kapag","kapwa","karamihan","katiyakan","katulad","kaya","kaysa","ko","kong","kulang","kumuha","kung","laban","lahat","lamang","likod","lima","maaari","maaaring","maging","mahusay","makita","marami","marapat","masyado","may","mayroon","mga","minsan","mismo","mula","muli","na","nabanggit","naging","nagkaroon","nais","nakita","namin","napaka","narito","nasaan","ng","ngayon","ni","nila","nilang","nito","niya","niyang","noon","o","pa","paano","pababa","paggawa","pagitan","pagkakaroon","pagkatapos","palabas","pamamagitan","panahon","pangalawa","para","paraan","pareho","pataas","pero","pumunta","pumupunta","sa","saan","sabi","sabihin","sarili","sila","sino","siya","tatlo","tayo","tulad","tungkol","una","walang"]

In [6]:
nltk.download('stopwords')
from nltk.corpus import stopwords
taglish = stopwords.words('english') + stopwords_filipino
print(taglish)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Winzyl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer


def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    """
    stemmer = PorterStemmer()
    stopwords_english = taglish
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove links
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    tweet = re.sub(r'#', '', tweet)
    # tokenize
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and provide data by looping all tweets
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [8]:
all_positive_tweets = pos
all_negative_tweets = neg

In [9]:
print(len(all_positive_tweets))
print(len(all_negative_tweets))

564
1039


In [10]:
test_pos = all_positive_tweets[400:]
train_pos = all_positive_tweets[:400]
test_neg = all_negative_tweets[800:]
train_neg = all_negative_tweets[:400]


train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [11]:
print(len(test_pos))
print(len(train_pos))
print(len(test_neg))
print(len(train_neg))

164
400
239
400


In [12]:
print(process_tweet(train_x[4]))

['lectur', 'review', 'school', 'think', 'much', 'daw', 'result', 'board', 'exam', 'kasi', 'time', 'take', "they'll", 'make', 'sure', 'kayang', 'yung', 'board', 'huhuhuhu', 'feel', 'like', 'realli', 'good', 'choic', 'mag', 'enrol', 'undergrad', 'review']


In [13]:
print(train_x)

["pero it's okay lang gihapon kay mano mano man sa board exam puhon puhon https://t.co/0quuKz01jm", '@Mahalima_19 @SB19Official Ako board exam for teachers ang inaantay ko di kasi natuloy yung dito sa NCR eh.\n\nYes nakakaproud mga bagong CPA, laban lang din para sa iba.', '@Readonlymemor17 Kunting push ka nalang po. Ikaw naman po next na papasa sa board exam ??', '@cosmicchilld @SB19Official Taking up BSA ka ba kaps? Kaya mo yan, basta ang goal mo is hindi lang makapasa ng course. End goal in mind ka pa din (board exam) para relaxed ka na pag review. ?\n\nVoteSB19 onRAWR \n@SB19Official \n#SB19SquidBreak\n\n#RAWRAwards2021 #RAWRATIN', "sabi ng lecturer namin sa review school, we don't have to think too much daw sa result ng board exam kasi when its our time to take it na, they'll make sure na kayang kaya namin yung board huhuhuhu i feel like it was really a good choice na mag enroll sa undergrad review nila", '@JodiSolid Laki ang income mga CPA marse isang libro lng yan ng book of acc

In [14]:
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [15]:
print(train_y)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.

In [16]:
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (800, 1)
test_y.shape = (403, 1)


In [17]:
freqs = build_freqs(train_x, train_y)

print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 2530


In [18]:
print(freqs)

{('okay', 1.0): 3, ('lang', 1.0): 36, ('gihapon', 1.0): 1, ('kay', 1.0): 9, ('mano', 1.0): 2, ('man', 1.0): 5, ('board', 1.0): 55, ('exam', 1.0): 59, ('puhon', 1.0): 2, ('teacher', 1.0): 1, ('inaantay', 1.0): 1, ('di', 1.0): 10, ('kasi', 1.0): 12, ('natuloy', 1.0): 1, ('yung', 1.0): 31, ('ncr', 1.0): 1, ('eh', 1.0): 2, ('ye', 1.0): 2, ('nakakaproud', 1.0): 1, ('bagong', 1.0): 2, ('cpa', 1.0): 11, ('kunt', 1.0): 1, ('push', 1.0): 2, ('nalang', 1.0): 23, ('po', 1.0): 10, ('naman', 1.0): 16, ('next', 1.0): 8, ('papasa', 1.0): 4, ('take', 1.0): 9, ('bsa', 1.0): 1, ('ba', 1.0): 7, ('kap', 1.0): 1, ('mo', 1.0): 25, ('yan', 1.0): 3, ('basta', 1.0): 3, ('goal', 1.0): 3, ('makapasa', 1.0): 6, ('cours', 1.0): 2, ('end', 1.0): 1, ('mind', 1.0): 2, ('relax', 1.0): 4, ('pag', 1.0): 15, ('review', 1.0): 7, ('votesb', 1.0): 1, ('19', 1.0): 1, ('onrawr', 1.0): 1, ('sb19squidbreak', 1.0): 1, ('rawraward', 1.0): 5, ('2021', 1.0): 9, ('rawratin', 1.0): 2, ('lectur', 1.0): 1, ('school', 1.0): 3, ('think',

In [19]:
print('raw data: \n', train_x[2])
print('\nprocessed data: \n', process_tweet(train_x[2]))

raw data: 
 @Readonlymemor17 Kunting push ka nalang po. Ikaw naman po next na papasa sa board exam ??

processed data: 
 ['kunt', 'push', 'nalang', 'po', 'naman', 'po', 'next', 'papasa', 'board', 'exam']


In [20]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # calculate the sigmoid of z
    h = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return h

In [21]:
if (sigmoid(0) == 0.5):
    print('works')
else:
    print('err')

if (sigmoid(4.92) == 0.9927537604041685):
    print('works')
else:
    print('err')

works
works


In [22]:
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2

9.210340371976294

In [23]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = -1./m * (np.dot(y.transpose(), np.log(h)) + np.dot((1-y).transpose(),np.log(1-h)))
        print(J)
        # update the weights theta
        theta = theta - (alpha/m) * np.dot(x.transpose(),(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta


In [43]:
# using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost for training is {tmp_J:.8f}.")
print(f"The resulting vector of weight is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

[[0.69314718]]
[[0.69305407]]
[[0.69296151]]
[[0.6928695]]
[[0.69277802]]
[[0.69268708]]
[[0.69259667]]
[[0.69250679]]
[[0.69241743]]
[[0.69232859]]
[[0.69224027]]
[[0.69215246]]
[[0.69206515]]
[[0.69197836]]
[[0.69189206]]
[[0.69180626]]
[[0.69172096]]
[[0.69163614]]
[[0.69155182]]
[[0.69146797]]
[[0.69138461]]
[[0.69130172]]
[[0.6912193]]
[[0.69113736]]
[[0.69105588]]
[[0.69097486]]
[[0.6908943]]
[[0.6908142]]
[[0.69073455]]
[[0.69065535]]
[[0.69057659]]
[[0.69049828]]
[[0.69042041]]
[[0.69034297]]
[[0.69026597]]
[[0.6901894]]
[[0.69011325]]
[[0.69003753]]
[[0.68996223]]
[[0.68988734]]
[[0.68981287]]
[[0.68973881]]
[[0.68966516]]
[[0.68959192]]
[[0.68951908]]
[[0.68944664]]
[[0.68937459]]
[[0.68930294]]
[[0.68923168]]
[[0.68916081]]
[[0.68909032]]
[[0.68902022]]
[[0.6889505]]
[[0.68888115]]
[[0.68881218]]
[[0.68874358]]
[[0.68867534]]
[[0.68860748]]
[[0.68853998]]
[[0.68847284]]
[[0.68840605]]
[[0.68833963]]
[[0.68827355]]
[[0.68820783]]
[[0.68814245]]
[[0.68807742]]
[[0.68801274]]
[

In [25]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [26]:
#test 1
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[  1. 176. 314.]]


In [27]:
# test 2
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]


In [28]:
print(np.zeros((3, 1)))

[[0.]
 [0.]
 [0.]]


In [42]:
X = np.zeros((len(train_x), 3))#array structure
for i in range(len(train_x)):#loop through train x to extract features on each
    X[i, :]= extract_features(train_x[i], freqs)

Y = train_y # training labels

J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

[[0.69314718]]
[[0.69314569]]
[[0.69314419]]
[[0.6931427]]
[[0.6931412]]
[[0.69313971]]
[[0.69313821]]
[[0.69313672]]
[[0.69313523]]
[[0.69313373]]
[[0.69313224]]
[[0.69313074]]
[[0.69312925]]
[[0.69312775]]
[[0.69312626]]
[[0.69312477]]
[[0.69312327]]
[[0.69312178]]
[[0.69312028]]
[[0.69311879]]
[[0.69311729]]
[[0.6931158]]
[[0.69311431]]
[[0.69311281]]
[[0.69311132]]
[[0.69310982]]
[[0.69310833]]
[[0.69310684]]
[[0.69310534]]
[[0.69310385]]
[[0.69310235]]
[[0.69310086]]
[[0.69309937]]
[[0.69309787]]
[[0.69309638]]
[[0.69309488]]
[[0.69309339]]
[[0.6930919]]
[[0.6930904]]
[[0.69308891]]
[[0.69308742]]
[[0.69308592]]
[[0.69308443]]
[[0.69308293]]
[[0.69308144]]
[[0.69307995]]
[[0.69307845]]
[[0.69307696]]
[[0.69307547]]
[[0.69307397]]
[[0.69307248]]
[[0.69307098]]
[[0.69306949]]
[[0.693068]]
[[0.6930665]]
[[0.69306501]]
[[0.69306352]]
[[0.69306202]]
[[0.69306053]]
[[0.69305904]]
[[0.69305754]]
[[0.69305605]]
[[0.69305456]]
[[0.69305306]]
[[0.69305157]]
[[0.69305008]]
[[0.69304858]]
[[0

In [30]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred =sigmoid(np.dot(x,theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [31]:
for tweet in ['ako ay masayang masaya', 'ikaw ay masama', 'talgang natuwa ako', 'salamat', 'salamat salamat', 'salamat salamat salamat']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

ako ay masayang masaya -> 0.500008
ikaw ay masama -> 0.499994
talgang natuwa ako -> 0.500000
salamat -> 0.500021
salamat salamat -> 0.500042
salamat salamat salamat -> 0.500064


In [32]:
positive = 'masaya ako ngayon'
predict_tweet(positive, freqs, theta)

array([[0.50000766]])

In [33]:
negative = 'galit ako ngayon'
predict_tweet(negative, freqs, theta)

array([[0.49997453]])

In [34]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
    print(f"Total True Prediction: {(y_hat==np.squeeze(test_y)).sum()}")
    print(f"total number of predictions: {len(test_x)}")

    ### END CODE HERE ###
    
    return accuracy

In [35]:
def test_precision(test_x, test_y, freqs, theta):
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    x_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5 and tweet in test_pos: #true positive
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5 and tweet in test_pos: #true positive
            x_hat.append(1)
        
        if y_pred > 0.5 and tweet not in test_pos: #false positive
            x_hat.append(1)

        else:
            # append 0 to the list
            x_hat.append(0)
            
    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    presision = (sum(y_hat)/sum(x_hat))
    print(f"TRUE positives: {sum(y_hat)}")
    print(f"total predicted positives: {sum(x_hat)}")

    ### END CODE HERE ###
    
    return presision

In [36]:
def test_recall(test_x, test_y, freqs, theta):
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    x_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5 and tweet in test_pos: #true positive
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5 and tweet in test_pos: #true positive
            x_hat.append(1)
        
        if y_pred < 0.5 and tweet not in test_neg: #false negative
            x_hat.append(1)

        else:
            # append 0 to the list
            x_hat.append(0)
            
    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    recall = (sum(y_hat)/sum(x_hat))
    print(f"TRUE positives: {sum(y_hat)}")
    print(f"total actual positive: {sum(x_hat)}")

    ### END CODE HERE ###
    
    return recall

In [37]:
tmp_recall = test_recall(test_x, test_y, freqs, theta)
print(f"Logistic regression model's recall = {tmp_recall:.4f}")

TRUE positives: 41
total actual positive: 164
Logistic regression model's recall = 0.2500


In [38]:
tmp_precision = test_precision(test_x, test_y, freqs, theta)
print(f"Logistic regression model's precision = {tmp_precision:.4f}")

TRUE positives: 41
total predicted positives: 59
Logistic regression model's precision = 0.6949


In [39]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Total True Prediction: 262
total number of predictions: 403
Logistic regression model's accuracy = 0.6501


In [40]:
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('Raw data:', x)
        print('Processed data:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
Raw data: Salamat naman after 5 months nakakuha din ng schedule for vaccine
Processed data: ['salamat', 'naman', '5', 'month', 'nakakuha', 'schedul', 'vaccin']
1	0.49969286	b'salamat naman 5 month nakakuha schedul vaccin'
Raw data: Gusto ko lang naman mag pa vaccine!!!!!! Punyeta
Processed data: ['lang', 'naman', 'mag', 'vaccin', 'punyeta']
1	0.49949927	b'lang naman mag vaccin punyeta'
Raw data: hindi naman masakit mag vaccine kaya magpa vaccine na kayo ppl <33
Processed data: ['naman', 'masakit', 'mag', 'vaccin', 'magpa', 'vaccin', 'kayo', 'ppl', '<3', '3']
1	0.49984633	b'naman masakit mag vaccin magpa vaccin kayo ppl <3 3'
Raw data: Abi nakog di malihok ang bukton after vaccine hahah pero goods raman diay
Processed data: ['abi', 'nakog', 'di', 'malihok', 'bukton', 'vaccin', 'hahah', 'good', 'raman', 'diay']
1	0.49949335	b'abi nakog di malihok bukton vaccin hahah good raman diay'
Raw data: Covid Vaccine today. Hoping na maging goods lahat and wala ng problem na m

In [41]:
tweet = 'online class'
print(process_tweet(tweet))
y_hat = predict_tweet(tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['onlin', 'class']
[[0.4999304]]
Negative sentiment


In [46]:
test2 = 
print(process_tweet(test2))

SyntaxError: unterminated string literal (detected at line 1) (3056314329.py, line 1)